In [91]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from geopy.geocoders import Nominatim
# import geopy.distance
# from geopy.extra.rate_limiter import RateLimiter
# import json
# import 
# import os
# import folium
# import geopandas as gpd
# import fiona
# import shapefile as shp
# from difflib import SequenceMatcher
# from shapely.geometry import Point, Polygon
# from geopandas import GeoDataFrame

In [383]:
import folium
import numpy as np
from sklearn.preprocessing import minmax_scale
from collections import defaultdict, OrderedDict
import geopandas as gpd
import pandas as pd
from folium.plugins import HeatMapWithTime

In [378]:
member_use=pd.read_csv("member_usage_start")
member_use
member_use.drop(["Unnamed: 0"],axis=1,inplace=True)
member_use.rename(columns={"0":"count"},inplace=True)
member_use["count"]=minmax_scale(member_use["count"])
casual_use=pd.read_csv("casual_usage_start")
casual_use.drop(["Unnamed: 0"],axis=1,inplace=True)
casual_use.rename(columns={"0":"count"},inplace=True)
casual_use["count"]=minmax_scale(casual_use["count"])
#Dictionary for {month:(station,usage)} for 2011
member_heat=defaultdict(list)
for r in member_use.itertuples():
    member_heat[r.month].append([r.lat,r.long,r.count])
member_heat_start=OrderedDict(sorted(member_heat.items()))
#Dictionary for {month:(station,usage)} for 2012
casual_heat=defaultdict(list)
for r in casual_use.itertuples():
    casual_heat[r.month].append([r.lat,r.long,r.count])
casual_heat_start=OrderedDict(sorted(casual_heat.items()))

In [376]:
member_use=pd.read_csv("member_usage_end")
member_use
member_use.drop(["Unnamed: 0"],axis=1,inplace=True)
member_use.rename(columns={"0":"count"},inplace=True)
member_use["count"]=minmax_scale(member_use["count"])
casual_use=pd.read_csv("casual_usage_end")
casual_use.drop(["Unnamed: 0"],axis=1,inplace=True)
casual_use.rename(columns={"0":"count"},inplace=True)
casual_use["count"]=minmax_scale(casual_use["count"])
#Dictionary for {month:(station,usage)} for 2011
member_heat=defaultdict(list)
for r in member_use.itertuples():
    member_heat[r.month].append([r.lat,r.long,r.count])
member_heat_end=OrderedDict(sorted(member_heat.items()))
#Dictionary for {month:(station,usage)} for 2012
casual_heat=defaultdict(list)
for r in casual_use.itertuples():
    casual_heat[r.month].append([r.lat,r.long,r.count])
casual_heat_end=OrderedDict(sorted(casual_heat.items()))

In [ ]:
#Census information for DC
geo_dc=gpd.read_file(r"C:\Users\giorg\Desktop\Trips\map\Census_2010_DC.geojson")
pop_dc=[]
pop_dc=pd.DataFrame(pop_dc)
pop_dc["geoid"]=0
pop_dc["population"]=0
pop_dc["population"]=geo_dc["P0010001"]
pop_dc["geoid"]=geo_dc["TRACT"]
drop_list=["TRACT","geometry"]
geo_dc=geo_dc.drop(geo_dc.columns.difference(drop_list),axis=1)
geo_dc.rename(columns={"TRACT":"geoid"},inplace=True)
#Census information for Virginia. Due to the fact that population breakdown is in county level and not lower, a comparison with DC is not feasible
pop_virginia=pd.read_csv(r"C:\Users\giorg\Desktop\Trips\map\Census_2010_Virginia.csv")
pop_virginia.columns=pop_virginia.columns.str.replace(" ","_")
pop_virginia.rename(columns={"B01003_001E":"population","county_1":"geoid"},inplace=True)
pop_virginia=pop_virginia.loc[(pop_virginia["geoid"]==13)|(pop_virginia["county"]=="Alexandria city, Virginia")]
pop_virginia=pop_virginia.loc[(pop_virginia["year"]==2011)]
drop_list=["population","geoid"]
pop_virginia=pop_virginia.drop(pop_virginia.columns.difference(drop_list),axis=1)
pop_virginia["geoid"]=pop_virginia["geoid"].replace({13:51013,510:51510})
geo_virginia=gpd.read_file(r"C:\Users\giorg\Desktop\Trips\map\Land_Virginia.geojson")
geo_virginia["geoid"]=geo_virginia["geoid"].str[:5]
geo_virginia=geo_virginia.loc[(geo_virginia["geoid"]=="51013")|(geo_virginia["geoid"]=="51510")]
drop_list=["geoid","geometry"]
geo_virginia=geo_virginia.drop(geo_virginia.columns.difference(drop_list),axis=1)
#Metro network map
metro_lines=gpd.read_file(r"C:\Users\giorg\Desktop\Trips\map\Metro_lines.geojson")
#Park and ride stations
park=gpd.read_file(r"C:\Users\giorg\Desktop\Trips\map\Park_and_Ride_Points.geojson")

In [384]:
m=folium.Map(location=[38.900497,-77.007507],zoom_start=12,min_zoom=11,zoom_control=False,Control_scale=True)
fg1=folium.FeatureGroup(name="Metro Network",show=False)
fg4=folium.FeatureGroup(name="Park and Ride stations",show=False)
folium.Choropleth(geo_data=geo_dc,data=pop_dc,columns=["geoid","population"],key_on="feature.properties.geoid",
    fill_color="YlGnBu",name="Population density",fill_opacity=0.6,line_opacity=0.2,show=False).add_to(m)
m.add_child(fg1)
m.add_child(fg4)
HeatMapWithTime(data=list(member_heat_start.values()),index=list(member_heat_start.keys()),show=False,name="Member usage Starting stations").add_to(m)
HeatMapWithTime(data=list(casual_heat_start.values()),index=list(casual_heat_start.keys()),show=False,name="Casual usage Starting stations").add_to(m)
HeatMapWithTime(data=list(member_heat_end.values()),index=list(member_heat_end.keys()),show=False,name="Member usage Ending stations").add_to(m)
HeatMapWithTime(data=list(casual_heat_end.values()),index=list(casual_heat_end.keys()),show=False,name="Casual usage Ending stations").add_to(m)
for i in range(len(park)):
    fg4.add_child(folium.CircleMarker(location=[park.loc[i,"geometry"].y,park.loc[i,"geometry"].x],popup=park.loc[i,"NAME"],color="red",radius=5))
folium.GeoJson(r"C:\Users\giorg\Desktop\Trips\map\Metro_Lines.geojson").add_to(fg1)
folium.GeoJson(r"C:\Users\giorg\Desktop\Trips\map\Metro_Stations.geojson").add_to(fg1)
folium.LayerControl().add_to(m)
m